This is the Deepnote File 

In [1]:
#We all ready?


# 

In [7]:
import pandas as pd
import numpy as np
from scipy import stats


# Load Dataset

In [3]:
#Build a Dataloader function
from pandas import read_csv
from numpy import dstack

def dataloader(d_path):
	dataframe = read_csv(d_path, header=None, delim_whitespace=True)
	return dataframe.values

data = dataloader('/work/CrispCerebella/Dataset/UCI HAR Dataset/train/Inertial Signals/total_acc_y_train.txt')

print(data.shape)

# function to load a group of data and create a 3 dimentional tensor out of it
def dataloader_group(filenames, prefix=''):
	loaded = list()
	for name in filenames:
		data = dataloader(prefix + name)
		loaded.append(data)
	# create a 3 dimentional tensor out of the individual data files
	loaded = dstack(loaded)
	return loaded

# load all acc data into a 3 dimentinal tensor 
acc_filenames = ['total_acc_x_train.txt', 'total_acc_y_train.txt', 'total_acc_z_train.txt']
total_acc_xyz = dataloader_group(acc_filenames, prefix='/work/CrispCerebella/Dataset/UCI HAR Dataset/train/Inertial Signals/')
print(total_acc_xyz.shape)

# create a train and test group out of the data
def datasetloader(group, prefix=''):
	filepath = prefix + group + '/Inertial Signals/'
	filenames = list()
	# filenames: total acceleration
	filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
	# filenames: body acceleration
	filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
	# filenames: body gyroscope
	filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
	# load X data (input)
	X = dataloader_group(filenames, filepath)
	# load output labels
	y = dataloader(prefix + group + '/y_'+group+'.txt')
	return X, y
 
# load all train
trainX, trainy = datasetloader('train', '/work/CrispCerebella/Dataset/UCI HAR Dataset/')
print("Shape of training Data:",trainX.shape, trainy.shape)
# load all test
testX, testy = datasetloader('test', '/work/CrispCerebella/Dataset/UCI HAR Dataset/')
print("Shape of test Data:",testX.shape, testy.shape)
 


(7352, 128)
(7352, 128, 3)
Shape of training Data: (7352, 128, 9) (7352, 1)
Shape of test Data: (2947, 128, 9) (2947, 1)


In [8]:
stats.describe(testX)

DescribeResult(nobs=2947, minmax=(array([[-0.2454728, -1.112589 , -1.0158   , ..., -2.352782 , -2.528342 ,
        -1.862215 ],
       [-0.2459132, -1.245918 , -1.106383 , ..., -2.344799 , -2.904903 ,
        -1.729082 ],
       [-0.2448548, -1.129734 , -1.092525 , ..., -2.218206 , -2.255857 ,
        -1.754057 ],
       ...,
       [-0.2459054, -1.055127 , -1.064231 , ..., -2.885222 , -2.19911  ,
        -1.413129 ],
       [-0.2450202, -1.073974 , -1.084465 , ..., -2.901259 , -2.265966 ,
        -1.771897 ],
       [-0.2442375, -1.005888 , -1.106173 , ..., -2.946578 , -1.917106 ,
        -2.116511 ]]), array([[1.964238 , 1.090686 , 0.9758336, ..., 1.862086 , 2.444461 ,
        1.786059 ],
       [2.055219 , 1.083054 , 0.9845997, ..., 1.576708 , 2.191394 ,
        1.491645 ],
       [2.016067 , 1.041681 , 0.9831551, ..., 1.625857 , 2.47324  ,
        1.324922 ],
       ...,
       [1.918973 , 1.011461 , 0.9765373, ..., 2.973073 , 2.501685 ,
        1.558125 ],
       [2.01327  , 1.041

# Prepare Dataset

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=0bc38299-d7dc-427d-847e-3f965a2d07ef' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>